<a href="https://colab.research.google.com/github/DrakeData/NLP-Disaster_Tweets/blob/main/NLP_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Disaster Tweets

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re

## Ingest
Donwloading the data from Kaggle's API

In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [5]:
! cp './drive/MyDrive/Colab Notebooks/API Keys/kaggle.json' ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle competitions download nlp-getting-started

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 66.1MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 133MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 22.4MB/s]


In [34]:
train_df = pd.read_csv('train.csv')

test_df = pd.read_csv('test.csv')

## EDA

In [35]:
# Look at random sample of 10
train_df.sample(10)

,id,keyword,location,text,target
2563,3676,destroy,NaN,@SarniamakChris @Hromadske @kasiadear33 how s...,1
5679,8103,rescued,Canada,10-Month-Old Baby Girl was Rescued by Coastgua...,1
896,1297,bloody,65,@zhenghxn i tried 11 eyes akame ga kill and to...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
6580,9418,survivors,Texas,Violators of the new improved Reddit will be s...,0
1240,1787,buildings%20on%20fire,taken by piper curda,someone: mentions gansey on fire\nme busting t...,1
4454,6337,hostages,"Las Vegas, NV",#hot C-130 specially modified to land in a st...,1
4529,6440,injured,"Piedmont Triad, NC",Unlicensed teen driver among 2 killed in NC cr...,1
2034,2920,danger,"St Paul, MN",Training grains of wheat to bare gold in the A...,0
1190,1715,bridge%20collapse,Leicester,Leicester_Merc : ICYMI - #Ashes 2015: Australi...,0


Column notes:
- id: tweet unique identifier
- keyword: keyword from the tweet; can return blank
- location: location that the tweet was sent from; can return blank
- text: text from tweet
- target: labels the tweet if the tweet is about a real disaster (1) or not (0)

### Preprocessing
- Make all text lowercase
- Remove special characters
- Remove numbers 
- Remove stop words
- Remove extra spacing (whitespace)
- Remove weblinks
- Expand contractions when possible
- Tokenize


In [42]:
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
  sentence=str(sentence)
  sentence = sentence.lower()
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(rem_num)  
  filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  stem_words=[stemmer.stem(w) for w in filtered_words]
  lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  return " ".join(filtered_words)

text_ls = ['keyword', 'location', 'text']

for col in text_ls:
  for i in range(len(train_df[col])):
    train_df[col][i] = preprocess(train_df[col][i])

train_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,keyword,location,text,target
0,1,nan,nan,deeds reason earthquake may allah forgive,1
1,4,nan,nan,forest fire near ronge sask canada,1
2,5,nan,nan,residents asked shelter place notified officer...,1
3,6,nan,nan,people receive wildfires evacuation orders cal...,1
4,7,nan,nan,got sent photo ruby alaska smoke wildfires pou...,1


In [45]:
train_df.sample(10)

,id,keyword,location,text,target
4561,6484,injured,india,udhampur terror attack militants attack police...,1
3525,5039,eyewitness,bakersfield california,wake kern county eyewitness news mornings airi...,1
3632,5181,fatalities,washington,saudi arabia abha fatalities reported followin...,1
5814,8297,rubble,nan,accionempresa china ûªs stock market crash sum...,0
1355,1956,burning buildings,nan,yes gays totally destroying america see buildi...,0
5823,8314,rubble,london,wisenews china stock market crash gems rubble,1
6174,8805,sirens,nan,yay sirens,0
5151,7345,obliterate,dover,dicehateme puppyshogun makes sense paper beats...,0
2521,3623,desolation,nan,free kindle book aug thriller desolation run j...,0
4177,5933,harm,los angeles california,quality metrics penalties may harm patient car...,0
